# C.1. Assignment

## Module C: Options

### DRW New-Hire Learning Program

#### Summer 2022

Questions?
* Contact: Mark Hendricks
* hendricks@uchicago.edu


### Instructions

* Work on this individually or with your colleagues. No assigned groups.
* We encourage you to do more on the problems in the afternoon/evening.

# Context

Use the same interest rate tree and bond-price tree shown in the C.1. Notes, and used in pricing the interest rate floor derivative.

# Pricing a Swap

Consider a single-period swap that pays at time period 1 (t=0.5), the expiration payoff (and thus terminal value) is
$$\frac{100}{2}(r_1 −c)$$
* with $c=2\%$
* payments are semiannual

Take the viewpoint of a fixed-rate payer, floating rate receiver.

### 1.1 
What is the replicating trade using the two bonds (period 1 and period 2) as above?

### 1.2 
What is the price of the swap?

### 1.3 
What is the risk-neutral probability?

### 1.4 Using the swaps as the reference security
Use the period-2 swap (priced above) and the period-1 ($t=0.5$) bond to replicate the interest-rate option first discussed above, (the "put" on the interest rate.)

So instead of replicating the option with the two treasuries, now you're replicating/pricing it with a one-period bond and two-period swap.

Do you get the same answer?

# 2. Pricing a Call on a Bond

Try using the same tree to price a call on the period-2 bond, (1-year), at period 1 (6-months).
* Payoff = $\max(P_{1|2}-K,0)$
* Strike = \$99.00

### 2.1 
What is the replicating trade using the two bonds (period 1 and period 2) as above?

### 2.2 
What is the price of the option?

### 2.3 
What is the risk-neutral probability?

# **<span style="color:red">Solutions</span>**

In [1]:
import numpy as np
import pandas as pd

In [2]:
t_grid = [.5,1,1.5]
p_grid = [99.1338, 97.8925, 96.1531]
y_grid = [1.74, 2.13, 2.62]

quotes = pd.DataFrame({'price':p_grid, 'yield':y_grid},index=t_grid)
quotes.index.name = 'time'

In [3]:
P1_2 = np.array([98.3193, 99.5261])

FACE = 100

NFREQ = 2

r0 = .0174
r1 = np.array([.0339, .0095])
rK = .02

dt = 0.5

A = np.exp(r0*dt)
Z = np.exp(-r0*dt)

## **<span style="color:red">Solutions 1</span>**

In [4]:
v1swap = FACE * (r1-rK)/NFREQ
v1swap

array([ 0.695, -0.525])

In [5]:
beta = (v1swap[0] - v1swap[1]) / (P1_2[0] - P1_2[1])
alpha = (v1swap[0] - beta * P1_2[0])/FACE

v0swap = [alpha,beta] @ quotes['price'].iloc[0:2]
v0swap

0.2594912064434709

In [6]:
pstar = (A * quotes['price'].iloc[1] - P1_2[1])/(P1_2[0] - P1_2[1])
pstar

0.6448622316691163

In [7]:
Z * np.array([pstar,1-pstar])@v1swap

0.2594647314911574

### Swap-Based Option Pricing

In [8]:
fud = FACE * (rK-r1)
fud[fud<0] = 0
fud

array([0.  , 1.05])

In [9]:
beta = (fud[0] - fud[1]) / (v1swap[0] - v1swap[1])
alpha = (fud[0] - beta * v1swap[0])/FACE

f0 = [alpha,beta] @ np.array([quotes['price'].iloc[0],v0swap])
f0

0.3696419169953733

In [10]:
pstar = (A * v0swap - v1swap[1])/(v1swap[0] - v1swap[1])
pstar

0.6448841220701451

In [11]:
Z * np.array([pstar,1-pstar])@fud

0.36964175877587885

## Bond Call

In [12]:
K = 99
fud = P1_2-K
fud[fud<0] = 0
fud

array([0.    , 0.5261])

In [13]:
beta = (fud[0] - fud[1]) / (P1_2[0] - P1_2[1])
alpha = (fud[0] - beta * P1_2[0])/FACE

f0 = [alpha,beta] @ quotes['price'].iloc[0:2]
f0

0.18520820241072045

In [14]:
pstar = (A * quotes['price'].iloc[1] - P1_2[1])/(P1_2[0] - P1_2[1])
pstar

0.6448622316691163

In [15]:
Z * np.array([pstar,1-pstar])@fud

0.18521953991589396